In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import statsmodels

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'
df = pd.read_excel(data_path)

In [ ]:
df = df[df.Comment != 'Reanimation']

In [ ]:
df.head()

In [ ]:
# encode eineteilung
df['Einteilung_encoded'] = df['Einteilung (Hauptdiagnose)'].map({'Krankheit': 0, 'Unfall': 1})
df['Sverweis_Geschlecht_encoded'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})

Spo2 as continuous variable

univariate

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'SPO2 (Erstbefund)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['SPO2 (Erstbefund)']], distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

multivariate

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'SPO2 (Erstbefund)', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['SPO2 (Erstbefund)', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']], distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

Spo2 as categorical variable (< 86)

In [ ]:
df['spo2_cat'] = df['SPO2 (Erstbefund)'].apply(lambda x: 1 if x < 86 else 0)

univariate

In [ ]:
from statsmodels.stats.nonparametric import rank_compare_2indep

res = rank_compare_2indep(
    df[(df['spo2_cat'] == 1) & (df['GCS (Erstbefund)'].notna()) & (df['SPO2 (Erstbefund)'].notna())]['GCS (Erstbefund)'].values,
    df[(df['spo2_cat'] == 0) & (df['GCS (Erstbefund)'].notna()) & (df['SPO2 (Erstbefund)'].notna())]['GCS (Erstbefund)'].values)

In [ ]:
res.pvalue

multivariate

In [ ]:
temp_df = df[['GCS (Erstbefund)', 'spo2_cat', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['spo2_cat', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']], distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

graphical representation

In [ ]:
df['GCS (Erstbefund)'] = df['GCS (Erstbefund)'].astype('int')

In [ ]:
# plot spo2 vs gcs
fig, ax = plt.subplots()
sns.swarmplot(y='SPO2 (Erstbefund)', x='GCS (Erstbefund)', hue='spo2_cat', data=df, ax=ax)

ax.set_ylabel('First SpO2')
ax.set_xlabel('GCS')

# set legend title
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, ['< 86%', '>= 86%'], title='SpO2')



In [ ]:
# fig.savefig('/Users/jk1/Downloads/spo2_vs_gcs.png', dpi=600, bbox_inches='tight')